<a href="https://colab.research.google.com/github/bstrain71/usn_instruction_search/blob/master/usn_instruction_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### INGEST

In [0]:
%matplotlib inline

In [133]:
# Load gdrive
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [0]:
!pip install -q tika #install tika

In [0]:
from tika import parser

# Test code for seeing how tika works.
#raw = parser.from_file('/content/gdrive/My Drive/MSDS_422/OPNAV_Instructions/1000.16L With CH-2.pdf')
# raw['content'] gives the raw text of the pdf
#print(raw['content'])
#raw.keys()
#raw['metadata']['resourceName'] # Calls the file name - not metadata. not every file has metadata so don't rely on it.

In [0]:
import os

# The indices for these are the same so they can be zipped later.
# The .pdf filename is what is read, not the metadata, so whatever
# the file is called in the folder is what it will be listed as
# in the results.

documents = [] # Contains raw text from all .pdfs.
filenames = [] # Contains the filenames of the .pdfs

# Parse all .pdf contents and their filenames.
directory = '/content/gdrive/My Drive/MSDS_422/OPNAV_Instructions/'
for file in os.listdir(directory):
  temp = parser.from_file(directory+file)
  documents.append(temp['content'])
  filenames.append(temp['metadata']['resourceName'])

### EDA and Data Cleaning

Make everything in the document lowercase and remove stopwords. The stopwords list can be modified as needed, this one comes directly from the gensim documentation and is relatively standard.

In [0]:
from collections import defaultdict
from gensim import corpora

# improved list from Stone, Denis, Kwantes (2010)
STOPWORDS = """
a about above across after afterwards again against all almost alone along already also although always am among amongst amoungst amount an and another any anyhow anyone anything anyway anywhere are around as at back be
became because become becomes becoming been before beforehand behind being below beside besides between beyond bill both bottom but by call can
cannot cant co computer con could couldnt cry de describe
detail did didn do does doesn doing don done down due during
each eg eight either eleven else elsewhere empty enough etc even ever every everyone everything everywhere except few fifteen
fify fill find fire first five for former formerly forty found four from front full further get give go
had has hasnt have he hence her here hereafter hereby herein hereupon hers herself him himself his how however hundred i ie
if in inc indeed interest into is it its itself keep last latter latterly least less ltd
just
kg km
made make many may me meanwhile might mill mine more moreover most mostly move much must my myself name namely
neither never nevertheless next nine no nobody none noone nor not nothing now nowhere of off
often on once one only onto or other others otherwise our ours ourselves out over own part per
perhaps please put rather re
quite
rather really regarding
same say see seem seemed seeming seems serious several she should show side since sincere six sixty so some somehow someone something sometime sometimes somewhere still such system take ten
than that the their them themselves then thence there thereafter thereby therefore therein thereupon these they thick thin third this those though three through throughout thru thus to together too top toward towards twelve twenty two un under
until up unless upon us used using
various very very via
was we well were what whatever when whence whenever where whereafter whereas whereby wherein whereupon wherever whether which while whither who whoever whole whom whose why will with within without would yet you
your yours yourself yourselves
"""

# remove common words and tokenize
stoplist = set(STOPWORDS.split())
texts = [
    [word for word in document.lower().split() if word not in stoplist]
    for document in documents
]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

# Building the corpus.
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

### Modeling

Build the model to measure similarity between documents. Here we will use Latent Semantic Analysis (LSI)

In [0]:
from gensim import models
# LSI uses singular value decomposition - I think that
# num_topics corresponds do the number of singular values
# to use.
# "Indexing by Latent Semantic Analysis" <http://www.cs.bham.ac.uk/~pxt/IDA/lsa_ind.pdf>
# Latent Semantic Indexing <https://en.wikipedia.org/wiki/Latent_semantic_indexing>
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=200)

Now suppose a user typed in the query [whatever doc = 'user stuff']. We would like to sort the corpus documents in decreasing order of relevance to this query. Unlike modern search engines, here we only concentrate on a single aspect of possible similarities—on apparent semantic relatedness of their texts (words). No hyperlinks, no random-walk static ranks, just a semantic extension over the boolean keyword match:

In [140]:
# Here the 'doc' object is the user's search term.
# vec_bow cleans the user's search term
# vec_lsi is a list of tuples which are the similarities,

# This is only the boolean similarity!
# This is the same as doing Ctrl+F and counting
# the number of hits.

doc = "navy reserve"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]  # convert the query to LSI space
print(vec_lsi)

[(0, 0.20953591805203933), (1, -0.08371080856048055), (2, 0.3335065005276335), (3, 0.04844744519897767), (4, 0.20636374833524268), (5, -0.028140754621893485), (6, -0.036556940949814064), (7, 0.2038751622586038), (8, 0.11065184251086967), (9, -0.43559867237641514), (10, -0.17408789270377045), (11, 0.07901844817154288), (12, -0.1858362994981624), (13, -0.07761785022022237), (14, 0.09192836687309641), (15, -0.10322157257862406), (16, -0.14010313198102434), (17, 0.14741448566075813), (18, 0.23490911103842063), (19, -0.05123657702842499), (20, -0.09368114017994361), (21, 0.047366487481964904), (22, -0.16905245166066268), (23, 0.24136700933145136), (24, 0.03409186637205262), (25, 0.11158534957352377), (26, -0.2653334232762108)]


In addition, we will be considering `cosine similarity <http://en.wikipedia.org/wiki/Cosine_similarity>`_
to determine the similarity of two vectors. Cosine similarity is a standard measure
in Vector Space Modeling, but wherever the vectors represent probability distributions,
`different similarity measures <http://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence#Symmetrised_divergence>`_
may be more appropriate.

Initializing query structures
++++++++++++++++++++++++++++++++

To prepare for similarity queries, we need to enter all documents which we want
to compare against subsequent queries.



In [141]:
from gensim import similarities
index = similarities.MatrixSimilarity(lsi[corpus])  # transform corpus to LSI space and index it

<div class="alert alert-danger"><h4>Warning</h4><p>The class :class:`similarities.MatrixSimilarity` is only appropriate when the whole
  set of vectors fits into memory. For example, a corpus of one million documents
  would require 2GB of RAM in a 256-dimensional LSI space, when used with this class.

  Without 2GB of free RAM, you would need to use the :class:`similarities.Similarity` class.
  This class operates in fixed memory, by splitting the index across multiple files on disk, called shards.
  It uses :class:`similarities.MatrixSimilarity` and :class:`similarities.SparseMatrixSimilarity` internally,
  so it is still fast, although slightly more complex.</p></div>

Index persistency is handled via the standard :func:`save` and :func:`load` functions:



In [142]:
index.save('/tmp/deerwester.index')
index = similarities.MatrixSimilarity.load('/tmp/deerwester.index')

This is true for all similarity indexing classes (:class:`similarities.Similarity`,
:class:`similarities.MatrixSimilarity` and :class:`similarities.SparseMatrixSimilarity`).
Also in the following, `index` can be an object of any of these. When in doubt,
use :class:`similarities.Similarity`, as it is the most scalable version, and it also
supports adding more documents to the index later.

Performing queries
++++++++++++++++++

To obtain similarities of our query document against the nine indexed documents:



In [145]:
# index is the similarity index, or what we are checking against (corpus in the lsi space)
# vec_lsi is our query in the lsi space
# the output is the similarity tuples

sims = index[vec_lsi]

similarity_list = list(zip(sims, filenames))
#print(similarity_list)
print(sorted(similarity_list, reverse = True))

[(0.70106417, '1120.3B.pdf'), (0.65377223, '1120.12A.pdf'), (0.5841409, '1001.19D.pdf'), (0.45244828, '1120.10A.pdf'), (0.44439676, '1120.6.pdf'), (0.42569372, '1001.27.pdf'), (0.4212415, '1040.11D.pdf'), (0.41672656, '1110.1B.pdf'), (0.35443875, '1120.5B.pdf'), (0.33901176, '1001.26C.pdf'), (0.32533118, '1120.8A.pdf'), (0.319309, '1000.26B.pdf'), (0.2606853, '1000.24D.pdf'), (0.25301617, '1000.1N.pdf'), (0.24673752, '1120.13B.pdf'), (0.23311296, '1120.11A.pdf'), (0.2297713, '1001.20C.pdf'), (0.22353701, '1070.2C.pdf'), (0.21817255, '1120.9A OPNAV.pdf'), (0.20258094, '1160.6C.pdf'), (0.19712898, '1120.7A.pdf'), (0.19594117, '1120.4C.pdf'), (0.19505413, '1000.16L With CH-2.pdf'), (0.0969552, '1160.9A.pdf'), (0.095172174, '1160.8B.pdf'), (0.058286358, '1100.4C.pdf'), (0.027300106, '1100.5.pdf')]


### Conclusion

This model could be very useful in production. There is a huge corpus of instructions that USN personnel have to sort through in order to get information - an easy way to query this corpus to find relevant instructions given a topic would save everyone time and energy. There is no objective metric with which to measure 'accuracy' but the searches have consistently returned sane and sensible results during operator testing.